In [63]:
import json
import requests

In [64]:
import cloudscraper
import geopandas as gpd
import h3

In [65]:
from shapely.geometry import Point
from tqdm.notebook import tqdm

## Get State Data

In [78]:
file_path = "data/cb_2018_us_state_500k/cb_2018_us_state_500k.shp"
states_df = gpd.read_file(file_path)

In [79]:
states_df = states_df[["STUSPS", "NAME", "geometry"]]

In [108]:
country_shape = states_df["geometry"].union_all()

In [109]:
h3_shape = h3.geo_to_h3shape(country_shape)
h3_cells = h3.h3shape_to_cells(h3_shape, res=5)

## Get McDonalds Data

In [110]:
scraper = cloudscraper.create_scraper()

In [111]:
restaurant_list = []
for h3_cell in tqdm(h3_cells, desc="Parsing H3 Cells"):
    lat, lng = h3.cell_to_latlng(h3_cell)
    url = (
        "https://www.mcdonalds.com/googleappsv2/geolocation?latitude="
        + str(lat)
        + "&longitude="
        + str(lng)
        + "&radius=100&maxResults=1000&country=us&language=en-us"
    )
    locations = scraper.get(url).text
    if not locations:
        continue
    features = json.loads(locations)["features"]
    for feature in features:
        state_code = feature["properties"]["subDivision"]
        restaurant_list.append(
            {"STATE": state_code, "geometry": Point(feature["geometry"]["coordinates"])}
        )

Parsing H3 Cells:   0%|          | 0/36446 [00:00<?, ?it/s]

In [112]:
mcdonalds_gdf = gpd.GeoDataFrame(restaurant_list, crs=4326)

In [113]:
mcdonalds_new_gdf = mcdonalds_gdf.drop_duplicates().reset_index()[["STATE", "geometry"]]

In [114]:
mcdonalds_new_gdf.to_file("data/mcdonalds_va_h3_5.gpkg")